In [1]:
from pymongo import MongoClient
import pymongo
db_url= #
client = MongoClient(db_url)

# See all databases: list(client.list_databases())
airbnb=client['sample_airbnb']['listingsAndReviews']
geospatial=client['sample_geospatial']['shipwrecks']
netflix=client['sample_mflix']
movies=client['sample_mflix']['movies']
supplies=client['sample_supplies']['sales']
training=client['sample_training']
companies=client['sample_training']['companies']
weather=client['sample_weatherdata']['data']
grades = training['grades'] 

# OTHER NOTES --- 
# list dbs
# list_databases()

# list collections
# list_collection_names()

# SORT --> SKIP --> LIMIT
# collection.distinct('address.country')


# Query construction
# collection.find() 
# collection.count_documents()


### All Query Operators discussed below can be found here:
https://docs.mongodb.com/manual/reference/operator/query/

### All questions are worth 4 points

## Problem 1

Find a document in the movies collection where all the following are true:

~~~
1) "genres" is not in the following: "Drama","Fantasy". Use the NOT operator.
2) "countries" is in "USA","Spain" OR "languages" is "English"
3) "imdb.rating" is greater than 7 OR "tomatoes.viewer.rating" is greater than 4
4) Limit to 1 result
5) Sort by year descending and title descending
6) Return the following fields: "_id,"genres", "countries", "languages","imdb", "tommatoes", "title"
~~~

In [2]:
list(movies
     .find({"$and":[
         {"genres":{"$not": {"$in":["Drama","Fantasy"]}}},
         {"$or": [{"countries":{"$in": ["USA","Spain"]}},
                  {"languages":"English"}]},
            {"$or": [{"imdb.rating":{"$gt": 7}},
                     {"tomatoes.viewer.rating":{"gt":4}}]}
     ]},{"_id":1, "genres":1, "countries":1, "languages":1, "imdb":1, "tommatoes":1, "title":1})
     .sort([("year",-1),("title",-1)])
     .limit(1)
)

[{'_id': ObjectId('573a13eaf29313caabdcfbc1'),
  'genres': ['Documentary'],
  'countries': ['USA'],
  'title': 'The Roosevelts: An Intimate History',
  'languages': ['English'],
  'imdb': {'rating': 8.8, 'votes': 682, 'id': 3400010}}]

## Problem 2

Find a document in the companies collection where:

~~~
1) "founded_year" is in the following: 2006, 2008, 20012
2) Where not any "is_past" field in the array of embedded objects "relationships" field is not equal to False. Also the "relationships" field must exist.
3) Return only the relevant fields in the query criteria - "relationships", "founded_year" and "_id".
4) Limit to 1 result
5) Sort on "founded_year" in ascending order
~~~

In [4]:
list(companies
     .find({"founded_year": {"$in": [2006, 2008, 2012]},
            "relationships.is_past": {"$not":{"$eq":False},"$exists":True}
     },
     {"relationships":1, "founded_year":1, "_id":1}
     )
     .sort("founded_year",1)
     .limit(1)
)

[{'_id': ObjectId('52cdef7c4bab8bd675297dae'),
  'founded_year': 2006,
  'relationships': [{'is_past': True,
    'title': 'Founder and CEO',
    'person': {'first_name': 'Ian',
     'last_name': 'Clarke',
     'permalink': 'ian-clarke'}},
   {'is_past': True,
    'title': 'Board',
    'person': {'first_name': 'Ian',
     'last_name': 'Clarke',
     'permalink': 'ian-clarke'}}]}]

## Problem 3

Find documents in the airbnb where:
~~~
1) the “number_of_reviews” is greater than 100
2) "beds" is greater than 5 OR "review_scores.review_scores_rating" is greater than 97
3) "host.host_is_superhost" is set to True OR "property_type" is a Villa
4) Limit to 1 result
5) Sort on "beds" in descending order
~~~

In [4]:
list(airbnb
     .find({"$and":
            [{"number_of_reviews": {"$gt": 100}},
             {"$or":[{"beds": {"$gt": 5}},
                     {"review_scores.review_scores_rating":{"$gt": 97}}]},
             {"$or":[{"host.host_is_superhost":True},
                     {"property_type": "Villa"}]}
            ]})
     .sort("beds",-1)
     .limit(1)
    
)

[{'_id': '1095174',
  'listing_url': 'https://www.airbnb.com/rooms/1095174',
  'name': 'Makai Hideaway',
  'summary': "This house is a hidden treasure. It is located one hour south of Kona and 65 miles  from Volcano National Park. It has fantastic ocean views and a wonderful deck.House is on an old lava flow -shop before you come as nearest  store is  22  minutes away!  12  minutes off highway 11 descend a curvy road with phenomenal ocean views.I think it's one of the best views on the island! Our house is 600 feet from the ocean! Walk to snorkeling/swimming hole in 3 minutes! There isn't a beach right by house.",
  'space': "A Hideaway quiet and peaceful, set in an authentic fishing village. Unwind and relax from city life and packaged vacations. Three bedroom, three baths, and game room. A large Lanai runs the length of the house for dolphin and whale watching. Have dinner on a beautiful Lanai and watch the incredible sunsets enjoy the warm climate with cool coastal breezes. One and 

## Problem 4

Find documents in the training.grades collection where:
~~~
1) "scores.score" array of embedded documents has no value greater than 80
2) Sort by "class_id" descending, student_id ascending
~~~

In [5]:
list(training.grades
     .find({"scores.score": {"$not": {"$gt": 80}}})
     .sort([("class_id", -1), ("student_id", 1)])
     .limit(1)
     )

[{'_id': ObjectId('56d5f7eb604eb380b0d8d9e6'),
  'student_id': 28.0,
  'scores': [{'type': 'exam', 'score': 0.2898348199689482},
   {'type': 'quiz', 'score': 13.88300815704696},
   {'type': 'homework', 'score': 61.998362329748836},
   {'type': 'homework', 'score': 35.69524705901108}],
  'class_id': 500.0}]

## Problem 5

Return the count of all the documents where:
~~~
1) "genres" not in the following: "Drama","Fantasy"
~~~

Write 2 different queries to return this count. 

In [6]:
method1 = movies.count_documents({"genres": {"$not": {"$in": ["Drama", "Fantasy"]}}})
method2 = movies.count_documents({"$and":[
    {"genres": {"$ne": "Drama"}},
    {"genres": {"$ne": "Fantasy"}}
    ]})

print(f'method 1 count: {method1}, method 2 count: {method2}')

method 1 count: 8993, method 2 count: 8993


## Problem 6

Return the documents from the airbnb collection where:
~~~
1) "amenities" field has values in: "TV", "Wifi"
2) "reviews" exists and is not equal to []
3) "property_type" is in one of the following values: "Hostel","Hotel" OR "room_type" is in one of the following values "Entire home/apt","Private room"
4) None of the fields in the array of embedded objects "reviews.reviewer_name" is not equal to "Theo" OR "reviews.reviewer_name" is not equal to "Marie" 
5) Only return the following "_id", "amenities", "reviews.reviewer_name","room_type","property_type", "name", "accommodates"
6) Sort by name descending, accommodates descending
7) Limit to 1 result
~~~

In [7]:
list(airbnb
     .find({"$and":[
         {"amenities":"TV"},
         {"amenities":"Wifi"},
         {"reviews": {"$exists":True, "$ne": []}},
         {"$or": [{"property_type": {"$in":["Hostel","Hotel"]}},
                  {"room_type": {"$in":["Entire home/apt","Private room"]}}
           ]},
           {"$or":[{"reviews.reviewer_name": {"$not": {"$eq": "Theo"}}},
                   {"reviews.reviewer_name": {"$not": {"$eq": "Marie"}}}
           ]}
     ]},
     {"_id":1, "amenities":1, "reviews.reviewer_name":1,"room_type":1,"property_type":1, "name":1, "accommodates":1}
     )
     .sort([("name", -1), ("accommodates", -1)])
     .limit(1)
)

[{'_id': '32369195',
  'name': '默默|sushe素舍  万象城/罗湖口岸/三地铁口/粤港美食/商旅首选一室一厅公寓',
  'property_type': 'Apartment',
  'room_type': 'Entire home/apt',
  'accommodates': 2,
  'amenities': ['TV',
   'Wifi',
   'Air conditioning',
   'Kitchen',
   'Pets allowed',
   'Elevator',
   'Washer',
   'Dryer',
   'Smoke detector',
   'Carbon monoxide detector',
   'First aid kit',
   'Fire extinguisher',
   'Essentials',
   'Shampoo',
   'Lock on bedroom door',
   'Hangers',
   'Hair dryer',
   'Iron',
   'Laptop friendly workspace',
   'Self check-in',
   'Smart lock'],
  'reviews': [{'reviewer_name': 'Chun Ki'}]}]

## Problem 7

In the movies collection:

~~~
1) "genres" not equal to Documentary
2) "imdb.rating" greater than 8
3) "tomatoes.viewer.rating" greater than 4
4) "rated" in one of the following values:"APPROVED","PG-13","PG" OR the "year" is in one of the following values:2000, 2008, 2010, 2012 OR "runtime" greater than 100
5) Only return "_id","genres", "imdb.rating", "tomatoes.viewer.rating", "rated","year","runtime"
6) Limit to 1 result
7) Sort by "runtime" ascending and "year" descending
~~~

In [8]:
list(movies
     .find({"$and":[
                    {"genres": {"$ne": "Documentary"}},
                    {"imdb.rating":{"$gt": 8}},
                    {"tomatoes.viewer.rating": {"$gt": 4}},
                    {"$or":[
                            {"rated": {"$in": ["APPROVED","PG-13","PG"]}},
                            {"year": {"$in": [2000, 2008, 2010, 2012]}},
                            {"runtime": {"$gt": 100}}
                    ]}
         
     ]},
     {"_id":1,"genres":1, "imdb.rating":1, "tomatoes.viewer.rating":1, "rated":1,"year":1,"runtime":1}
     )
     .sort([("runtime", 1),("year", -1)])
     .limit(1)
    
)

[{'_id': ObjectId('573a13a3f29313caabd0f030'),
  'genres': ['Animation', 'Short', 'Comedy'],
  'runtime': 3,
  'rated': 'G',
  'year': 2000,
  'imdb': {'rating': 8.2},
  'tomatoes': {'viewer': {'rating': 4.1}}}]

## Problem 8

Return the document from the companies collection where the following are true:
~~~
1) "number_of_employees" is greater than 10
2) One object in the "funding_rounds" array has a "funded_day" field value of 1, a "funded_month" between 9 and 12, and a "raised_amount" of greater than or equal to 1000000
3) One object in the "investments" array which is embedded in the "funding_rounds" array has a "financial_org.name" field value of "Sequoia Capital" and "financial_org.permalink" field value of "sequoia-capital"
4) Return the "number_of_employees", "name", and "funding_rounds"
5) Limit to 1 result
6) Sort by name ascending and number_of_employees descending
~~~

In [9]:
list(companies
     .find({
         "number_of_employees":{"$gt": 10},
         "funding_rounds": {"$elemMatch": {
             "funded_day":1,
             "funded_month": {"$gte":9, "$lte":12},
             "raised_amount": {"$gte": 1000000}
         }},
         "funding_rounds.investments": {"$elemMatch": {
             "financial_org.name":"Sequoia Capital",
             "financial_org.permalink":"sequoia-capital"
         }}
     },
     {"_id":0,"number_of_employees":1, "name":1, "funding_rounds":1}
     )
     .sort([("name", 1),("number_of_employees", -1)])
     .limit(1)
     )

[{'name': 'Aster Data Systems',
  'number_of_employees': 50,
  'funding_rounds': [{'id': 2324,
    'round_code': 'angel',
    'source_url': '',
    'source_description': '',
    'raised_amount': 1000000,
    'raised_currency_code': 'USD',
    'funded_year': 2005,
    'funded_month': 11,
    'funded_day': 1,
    'investments': [{'company': None,
      'financial_org': None,
      'person': {'first_name': 'David',
       'last_name': 'Cheriton',
       'permalink': 'david-cheriton'}},
     {'company': None,
      'financial_org': None,
      'person': {'first_name': 'Anand',
       'last_name': 'Rajaraman',
       'permalink': 'anand-rajaraman'}},
     {'company': None,
      'financial_org': None,
      'person': {'first_name': 'Ron',
       'last_name': 'Conway',
       'permalink': 'ron-conway'}},
     {'company': None,
      'financial_org': {'name': 'First Round Capital',
       'permalink': 'first-round-capital'},
      'person': None},
     {'company': None,
      'financial_org':

## Problem 9

Write and execute the following as a MongoDB query.
~~~
SELECT tomatoes.viewer.rating, awards.wins, imdb.rating, _id, title
FROM movies
WHERE awards.wins > 3 
OR imdb.rating BETWEEN 7.5 AND 9 AND tomatoes.viewer.rating > 3.5
ORDER BY title 
LIMIT 1
~~~

In [13]:
list(movies
     .find({"$or":[
                            {"awards.wins": {"$gt": 3}},
                            {"$and":[
                                {"imdb.rating": {"$gte": 7.5, "$lte": 9}},
                                {"tomatoes.viewer.rating": {"$gt": 3.5}}
                            ]}
     ]},
     {"tomatoes.viewer.rating":1, "awards.wins":1, "imdb.rating":1, "_id":1, "title":1}
          )
     .sort("title",1)
     .limit(1)
)

[{'_id': ObjectId('573a13dff29313caabdb9439'),
  'imdb': {'rating': 7.2},
  'title': "'71",
  'tomatoes': {'viewer': {'rating': 3.9}},
  'awards': {'wins': 8}}]

## Problem 10

Write and execute the following as a MongoDB query.

~~~
SELECT _id, genres, runtime, type
FROM movies
WHERE (genres="Short" OR type="series") AND runtime < 30
ORDER BY runtime DESC 
LIMIT 1
~~~

In [11]:
list(movies
     .find({"$and":[
                    {"$or":[{"genres":"Short"},
                            {"type":"series"}]},
                    {"runtime": {"$lt": 30}}
     ]},
     {"_id":1, "genres":1, "runtime":1, "type":1})
     .sort("runtime", -1)
     .limit(1)
     )

[{'_id': ObjectId('573a1392f29313caabcd9bf8'),
  'genres': ['Comedy', 'Short'],
  'runtime': 29,
  'type': 'movie'}]